In [1]:
import pyhf
import pandas
import numpy as np
import altair as alt
# alt.renderers.enable('notebook')

/Users/lukasheinrich/Code/pyhfdev/dev/pyhfdevenv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/lukasheinrich/Code/pyhfdev/dev/pyhfdevenv/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
m = pyhf.simplemodels.hepdata_like([7],[20],[5])
d = [25] + m.config.auxdata

In [3]:
muscan = np.linspace(0,5,21)
results = [pyhf.infer.hypotest(mu,d,m,return_expected_set = True) for mu in muscan]

In [4]:
data = np.concatenate([
    muscan.reshape(-1,1),
    np.asarray([r[0] for r in results]).reshape(-1,1),
    np.asarray([r[1] for r in results]).reshape(-1,5)
],axis=1
)
df = pandas.DataFrame(data, columns = ['mu','obs'] + [f'exp_{i}' for i in range(5)])
df.head()

,mu,obs,exp_0,exp_1,exp_2,exp_3,exp_4
0,0.00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.25,0.837810,0.544269,0.671615,0.806411,0.920976,0.982722
2,0.50,0.726450,0.279981,0.428500,0.623443,0.825621,0.956105
3,0.75,0.640506,0.136181,0.259526,0.460995,0.717475,0.917542
4,1.00,0.512530,0.062690,0.149204,0.325505,0.602248,0.865054


In [5]:

band1 = alt.Chart(df).mark_area(opacity = 0.5,color='green').encode(
    x='mu',
    y='exp_1',
    y2='exp_3'
)

band2 = alt.Chart(df).mark_area(opacity = 0.5,color='yellow').encode(
    x='mu',
    y='exp_0',
    y2='exp_4'
)

line1 = alt.Chart(df).mark_line(color = 'black').encode(
    x='mu',
    y='obs'
)

line2 = alt.Chart(df).mark_line(color = 'black', strokeDash=[5,5]).encode(
    x='mu',
    y='exp_2'
)

nearest = alt.selection_single(
    nearest=True, on='mouseover',fields=['mu'], empty='none'
)


point = alt.Chart(df).mark_point(color = 'black').encode(
    x='mu',
    y='obs',
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
).add_selection(
    nearest
)

text = line1.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'obs', alt.value(' '))
)


band2 + band1 + line1 + line2 + point + text

alt.LayerChart(...)